In [1]:
import pandas as pd 
import os

In [2]:
# Definimos rutas

data_path = "../raw_data"

# Definimos nombres de archivos
fao_data_file_name = "FAOSTAT_data_7-4-2022(2).csv"

# Cargamos datos
encode = "ISO-8859-1"

fao_data = pd.read_csv(os.path.join(data_path, fao_data_file_name), encoding = encode)
m49_fao_countries = pd.read_json("https://data.apps.fao.org/catalog/dataset/1712bf04-a530-4d55-bb66-54949213985f/resource/b0c1d224-23ea-425d-b994-e15f76feb26b/download/m49-countries.json")

In [3]:
# Agregamos el ISO CODE 3 para todos los paises
cw_fao_names_iso_code3 = {i:j for i,j in zip(m49_fao_countries["country_name_en"], m49_fao_countries["ISO3"])}
fao_data["iso_code3"] = fao_data["Area"].replace(cw_fao_names_iso_code3)


In [4]:
# Cargamos el crosswalk entre Items de FAO ---> sisepuede
cw =  pd.read_csv(os.path.join(data_path, "items_classification.csv"))[["Item_Fao", "File_Sisepuede"]]

fao_data = fao_data[fao_data.Item.isin(cw.Item_Fao)]

cw_dict = {i : j for i,j in zip(cw["Item_Fao"], cw["File_Sisepuede"])}

# 30
fao_data["sisepuede_item"] = fao_data["Item"].replace(cw_dict)

fao_data = fao_data.groupby(["iso_code3", "Area","Year","sisepuede_item"])["Value"].mean().reset_index() 

fao_data = fao_data.pivot(index=['iso_code3', 'Area', 'Year'], columns='sisepuede_item', values='Value').reset_index()  

fao_data = fao_data.rename(columns = {"Area" : "Nation"})

fao_data["pop_lvst_initial_cattle_dairy"] = fao_data["pop_lvst_initial_cattle_nondairy"]

In [5]:

## Hacemos interpolación para cubrir los datos faltantes
## Y guardamos datos históricos

path_historical_data = "../input_to_sisepuede/historical"

for sise_var in [i for i in fao_data.columns if "pop_lvst_" in i]:

    if fao_data[sise_var].isna().any():   
        #print(f"{sise_var}    Tiene NaN")
        fao_data[sise_var] = fao_data.groupby(["Nation"])[sise_var].apply(lambda x: x.interpolate().fillna(method='bfill')).reset_index()[sise_var].fillna(0)


    fao_data[["iso_code3","Nation", "Year",sise_var]].to_csv(os.path.join(path_historical_data,f"{sise_var}.csv"), index = False)

/tmp/ipykernel_4896/2518583400.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  fao_data[sise_var] = fao_data.groupby(["Nation"])[sise_var].apply(lambda x: x.interpolate().fillna(method='bfill')).reset_index()[sise_var].fillna(0)
/tmp/ipykernel_4896/2518583400.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..

In [6]:
## Datos proyectados. El último año lo tomamos como constante para hasta 2050
max_year = fao_data.Year.max()

fao_data = fao_data.query(f"Year=={max_year}")

fao_data = fao_data.drop(columns = "Year")

df_year = pd.DataFrame({"Year" : range(max_year+1, 2051)})
fao_data_projected = fao_data.merge(df_year, how = "cross") 

path_projected_data = "../input_to_sisepuede/projected"

for sise_var in [i for i in fao_data.columns if "pop_lvst_" in i]:
    fao_data_projected[["iso_code3","Nation", "Year",sise_var]].to_csv(os.path.join(path_projected_data,f"{sise_var}.csv"), index = False)